In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.onnx
import onnx

from google.protobuf import json_format

dummy_input = Variable(torch.randn(2, 5))
model = torch.nn.Sequential(
          torch.nn.Linear(5, 3),
#           torch.nn.Dropout(p=0.9),
#           torch.nn.ReLU(),
#           torch.nn.Linear(3, 2),
#           torch.nn.Dropout(p=0.9),
          torch.nn.ReLU(),
#           torch.nn.Linear(2, 1),
#           torch.nn.Softmax(dim=1),
        )
torch.onnx.export(model, dummy_input, "model.proto", verbose=True)

graph(%1 : Float(2, 5)
      %2 : Float(3, 5)
      %3 : Float(3)) {
  %6 : Float(2, 3) = Gemm[alpha=1, beta=1, broadcast=1, transB=1](%1, %2, %3), uses = [%7.i0];
  %7 : Float(2, 3) = Relu(%6), uses = [%0.i0];
  return (%7);
}



In [2]:
# model = onnx.load("model.proto")
with open("model.proto", "rb") as f:
    s = f.read()
model = onnx.onnx_pb.ModelProto()
model.ParseFromString(s)
onnx.helper.strip_doc_string(model) # Cleaning

model_as_json = json_format.MessageToJson(model)
json_format.Parse(model_as_json, onnx.onnx_pb.ModelProto())
# print(model_as_json)
print(model)

ir_version: 2
producer_name: "pytorch"
producer_version: "0.3"
graph {
  node {
    input: "1"
    input: "2"
    input: "3"
    output: "6"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "broadcast"
      i: 1
      type: INT
    }
    attribute {
      name: "transB"
      i: 1
      type: INT
    }
  }
  node {
    input: "6"
    output: "7"
    op_type: "Relu"
  }
  name: "torch-jit-export"
  initializer {
    dims: 3
    dims: 5
    data_type: FLOAT
    name: "2"
    raw_data: "\024\357\232> D\374<\332\271\201>nc\322>P[\020\275\304R\227\275\010T$>\374\344\334\276\333\263\253\276^\337\334>\234\375\'>\214E\267\275\200:_;h\341\366\275\310@\221>"
  }
  initializer {
    dims: 3
    data_type: FLOAT
    name: "3"
    raw_data: "X0\233=`\004\213\274\375\037\\\276"
  }
  input {
    name: "1"
    type {
      tensor_type {
        elem_type: FL

In [3]:
import syft
import syft.nn
import syft.onnx
import imp
imp.reload(syft)
imp.reload(syft.nn)

import numpy as np
from syft import FloatTensor

dummy_input = FloatTensor([[1,2,3,4,5],[5,4,3,2,1]], autograd=True)
model = syft.nn.Sequential([
    syft.nn.Linear(5,3),
#     syft.nn.Dropout(p=0.9),
    syft.nn.ReLU(),
#     syft.nn.Linear(3, 2),
#     syft.nn.Dropout(p=0.9),
#     syft.nn.ReLU(),
#     syft.nn.Linear(2, 1),
#     syft.nn.Softmax(dim=1)
])
syft_model_as_json = syft.onnx.export(model, dummy_input, "model2.proto", verbose=True)
model_from_syft = onnx.onnx_pb.ModelProto()
json_format.Parse(syft_model_as_json, model_from_syft)
print(model_from_syft)

/Users/morgangiraud/.virtualenvs/openmined/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


ir_version: 2
producer_name: "openmined"
producer_version: "0.0.2"
domain: "org.openmined"
graph {
  node {
    input: "1"
    input: "2"
    input: "3"
    output: "7"
    name: "linear"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "broadcast"
      i: 1
      type: INT
    }
    attribute {
      name: "transB"
      i: 1
      type: INT
    }
  }
  node {
    input: "7"
    output: "8"
    op_type: "relu"
  }
  initializer {
    dims: 2
    dims: 5
    data_type: FLOAT
    float_data: 1.0
    float_data: 2.0
    float_data: 3.0
    float_data: 4.0
    float_data: 5.0
    float_data: 5.0
    float_data: 4.0
    float_data: 3.0
    float_data: 2.0
    float_data: 1.0
    name: "1"
  }
  initializer {
    dims: 5
    dims: 3
    data_type: FLOAT
    float_data: -0.369104147
    float_data: -0.273278654
    float_data: 0.374961078
    float_